<a href="https://colab.research.google.com/github/SamuelaAnastasi/PrivateAi_Challenge_FederatedLearning/blob/master/PrivateAi_Challenge_FederatedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Federated Learning
Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.
In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extension of PyTorch called PySyft.

In [1]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 276kB 42.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 28026 (delta 73), reused 82 (delta 46), pack-reused 27906
Receiving objects: 100% (28026/28026), 31.85 MiB | 20.95 MiB/s, done.
Resolving deltas: 100% (18494/18494), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |████████████████████████████████| 389kB 2.8MB/s 
ERROR: syft 0.1.21a1 has requirement msgpack>=0.6.1, but you'll have msgpac

In [0]:
import torch as th

In [3]:
x = th.tensor([1,2,3,4])
x

tensor([1, 2, 3, 4])

In [4]:
y = x + x
y

tensor([2, 4, 6, 8])

In [0]:
import syft as sy

In [0]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4])

tensor([1, 2, 3, 4])

#Basic Remote Execution in PySyft
The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning. 

To do this we need to work with pointers to tensors by creating a "pretend" machine owned by a "pretend" person - Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [10]:
# check bob objects for the moment is empty
bob._objects

{}

In [11]:
# create tensor
x = th.tensor([1,2,3,4])
x

tensor([1, 2, 3, 4])

In [0]:
# send tensors to bob and get a pointer to them
x = x.send(bob)

In [13]:
## check bob objects we passed
bob._objects

{39471476934: tensor([1, 2, 3, 4])}

In [14]:
x.location

<VirtualWorker id:bob #objects:1>

In [15]:
x.id_at_location

39471476934

In [16]:
x.id

70091292613

In [17]:
x.owner

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:70091292613 -> bob:39471476934]

In [19]:
# getting back objects from bob
x = x.get()
x

tensor([1, 2, 3, 4])

In [20]:
# check bob objects now is again empty
bob._objects

{}

##Project: Playing with Remote Tensors
Create another VirtualWorker called alice - send() and .get() a tensor to TWO workers

In [0]:
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.tensor([1,2,3,4])

In [0]:
# send tensor to both workers
x_pointer = x.send(bob, alice)

In [36]:
# Multi-Pointer object
x_pointer

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:63584750428 -> bob:76407714038]
	-> (Wrapper)>[PointerTensor | me:27138820338 -> alice:12708511673]

In [37]:
# Multi-Pointer's children
x_pointer.child.child

{'alice': (Wrapper)>[PointerTensor | me:27138820338 -> alice:12708511673],
 'bob': (Wrapper)>[PointerTensor | me:63584750428 -> bob:76407714038]}

In [25]:
bob._objects

{64043612998: tensor([1, 2, 3, 4])}

In [26]:
alice._objects

{32557939366: tensor([1, 2, 3, 4])}

In [27]:
#get tensors back
x_pointer.get()

[tensor([1, 2, 3, 4]), tensor([1, 2, 3, 4])]

In [28]:
bob._objects

{}

In [29]:
alice._objects

{}

In [0]:
# chain tensor creation and sending it to workers
x = th.tensor([1,2,3,4]).send(bob, alice)

In [31]:
bob._objects

{22471698515: tensor([1, 2, 3, 4])}

In [32]:
alice._objects

{88869673187: tensor([1, 2, 3, 4])}

In [33]:
# get and sum of tensors in the workers
x.get(sum_results=True)

tensor([2, 4, 6, 8])

##Lesson: Introducing Remote Arithmetic

In [0]:
# create tensors, send them to the same worker and perform sum on the remote tensors
x = th.tensor([1,2,3,4]).send(bob)
y = th.tensor([2,2,2,2]).send(bob)

In [39]:
x

(Wrapper)>[PointerTensor | me:57413361107 -> bob:21672544972]

In [40]:
y

(Wrapper)>[PointerTensor | me:1558866352 -> bob:49544249679]

In [41]:
# sum remote tensor creates another pointer to the remote sum tensor
z = x + y
z

(Wrapper)>[PointerTensor | me:89818952002 -> bob:84603547190]

In [42]:
z = z.get()
z

tensor([3, 4, 5, 6])

In [44]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:65729977352 -> bob:59539220483]

In [45]:
z = z.get()
z

tensor([3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4], requires_grad=True).send(bob)
y = th.tensor([2.,2,2,2], requires_grad=True).send(bob)

In [57]:
z = (x + y).sum()
z

(Wrapper)>[PointerTensor | me:33687032276 -> bob:15813503454]

In [58]:
z.backward()

(Wrapper)>[PointerTensor | me:51801139335 -> bob:22664361154]

In [59]:
x = x.get()
x

tensor([1., 2., 3., 4.], requires_grad=True)

In [60]:
x.grad

tensor([1., 1., 1., 1.])